In [ ]:
from __future__ import division, print_function
import os
import sys
from six.moves import cPickle as pickle

# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as pl
%matplotlib inline
import numpy as np

# Custom
import gary.dynamics as gd
import gary.integrate as gi
import gary.potential as gp
from gary.units import galactic

import ophiuchus.potential as op
from ophiuchus.data import OphiuchusData
from ophiuchus.util import integrate_forward_backward
from ophiuchus.coordinates import Ophiuchus
from ophiuchus import galactocentric_frame, vcirc, vlsr, RESULTSPATH
from ophiuchus.experiments import MockStreamGrid
from ophiuchus.util import get_potential_stream_prog
from ophiuchus.plot import surface_density

plotpath = "/Users/adrian/projects/ophiuchus-paper/figures/"
if not os.path.exists(plotpath):
    os.mkdir(plotpath)

In [ ]:
ophdata = OphiuchusData()

In [ ]:
all_names = ["static_mw"] + ["barred_mw_{}".format(i) for i in range(1,10)]
short_names = ["static"] + ["bar{}".format(i) for i in range(1,10)]
name_map = dict(zip(all_names, short_names))

## Mock streams

In [ ]:
# # name = 'static_mw'
# name = 'barred_mw_2'
# ix = 11

# # global style stuff
# style = dict(marker='o', ms=4, ls='none', alpha=0.25, color='#aaaaaa')
# data_style = dict(marker='o', ms=5, ls='none', ecolor='#666666', alpha=0.75)

# fig,axes = pl.subplots(2,1,figsize=(4,4.5),sharex=True,sharey='row')

# pot = op.load_potential(name)
# grid = MockStreamGrid.from_config(cache_path=os.path.join(RESULTSPATH, name, "mockstream"),
#                                   config_filename="../../global_mockstream.cfg",
#                                   potential_name=name)
# d = grid.read_cache()
# w = gd.CartesianPhaseSpacePosition.from_w(d['w'][ix].T, units=pot.units)
# model_c,model_v = w.to_frame(coord.Galactic, galactocentric_frame=galactocentric_frame, vcirc=vcirc, vlsr=vlsr)

# ix = distance_ix(model_c.l, model_c.b, model_c.distance)
# axes[0].plot(model_c.l[ix], model_c.b[ix], **style)
# axes[1].plot(model_c.l[ix], galactic.decompose(model_v[2][ix]), **style)

# _tmp = data_style.copy()
# _tmp.pop('ecolor')
# axes[0].plot(ophdata.coord.l.degree, ophdata.coord.b.degree, **_tmp)
# axes[1].errorbar(ophdata.coord.l.degree, ophdata.veloc['vr'].to(u.km/u.s).value, 
#                  ophdata.veloc_err['vr'].to(u.km/u.s).value, **data_style)

# axes[0].set_xlim(9,2)

# axes[0].set_ylabel("$b$ [deg]", fontsize=18)
# axes[0].set_ylim(26.5, 32.5)

# axes[1].set_ylabel(r"$v_r$ [${\rm km}\,{\rm s}^{-1}$]", fontsize=18)
# axes[1].set_ylim(225, 325)
# axes[1].set_xlabel("$l$ [deg]", fontsize=18)

# fig.tight_layout()

# fig.savefig("/Users/adrian/projects/talks/aas_ophiuchus_{}.png".format(name), dpi=400)

In [ ]:
style = {
    "linestyle": "none",
    "marker": 'o',
    "markersize": 3,
    "alpha": 0.05,
    "color": "#555555"
}
line_style = {
    "marker": None, 
    "linestyle": '--', 
    "color": 'k', 
    "linewidth": 1.5, 
    "dashes": (3,2)
}
levels = [-2,-1,0,1]

for k,name_group in enumerate([all_names[:5],all_names[5:]]):
    fig,allaxes = pl.subplots(3, 5, figsize=(9,6.5), sharex='col', sharey='row')
    for i,name in enumerate(name_group):
        axes = allaxes[:,i]
        
        # path to file to cache the likelihoods
        cache_file = os.path.join(RESULTSPATH, name, "mockstream", "ln_likelihoods.npy")
        lls = np.load(cache_file)
        best_ix = lls.sum(axis=1).argmax()
    
        pot, streams, prog = get_potential_stream_prog(name)
        stream = streams[:,best_ix]
        model_c,model_v = stream.to_frame(coord.Galactic, galactocentric_frame=galactocentric_frame, vcirc=vcirc, vlsr=vlsr)

#         ix = distance_ix(model_c.l, model_c.distance) & (model_c.l > 0) & (model_c.l < 10*u.deg)
        ix = (model_c.l > 0*u.deg) & (model_c.l < 11*u.deg) & (model_c.b > 25.5*u.deg) & (model_c.b < 34.5*u.deg)
        axes[0].plot(model_c.l[ix], model_c.b[ix], **style)
        axes[1].plot(model_c.l[ix], model_c.distance[ix], **style)
        axes[2].plot(model_c.l[ix], galactic.decompose(model_v[2][ix]), **style)
        
        # l,b
        axes[0].plot([3.8,3.8], [31.5,32.5], **line_style)
        axes[0].plot([5.85,5.85], [30.,31], **line_style)
        # l,d
        axes[1].plot([3.8,3.8], [8.7,9.3], **line_style)
        axes[1].plot([5.85,5.85], [7.2,7.8], **line_style)
        # l,vr
        axes[2].plot([3.8,3.8], [276,292], **line_style)
        axes[2].plot([5.85,5.85], [284,300], **line_style)
        
        # Axis limits
        axes[0].set_xlim(9.5,1.5)
        axes[0].set_ylim(26.1, 32.9)
        axes[1].set_ylim(5.3, 9.7)
        axes[2].set_ylim(225, 325)
        
        # Text
        axes[0].set_title(name_map[name], fontsize=20)
        axes[2].set_xlabel("$l$ [deg]", fontsize=18)
        if i == 0:
            axes[0].set_ylabel("$b$ [deg]", fontsize=18)
            axes[1].set_ylabel(r"$d_\odot$ [kpc]", fontsize=18)
            axes[2].set_ylabel(r"$v_r$ [${\rm km}\,{\rm s}^{-1}$]", fontsize=18)
    
    fig.tight_layout()
    
    fig.savefig(os.path.join(plotpath, "mockstream{}.pdf".format(k)))
    fig.savefig(os.path.join(plotpath, "mockstream{}.png".format(k)), dpi=400)

In [ ]:
corner_list

In [ ]:
fig,allaxes = pl.subplots(2, 5, figsize=(9,5), sharex=True, sharey=True)
for k,name_group in enumerate([all_names[:5],all_names[5:]]):
    for i,name in enumerate(name_group):
        ax = allaxes[k,i]
        
        # path to file to cache the likelihoods
        cache_file = os.path.join(RESULTSPATH, name, "mockstream", "ln_likelihoods.npy")
        lls = np.load(cache_file)
        best_ix = lls.sum(axis=1).argmax()
    
        pot, streams, prog = get_potential_stream_prog(name)
        stream = streams[:,best_ix]
        model_c,model_v = stream.to_frame(coord.Galactic, galactocentric_frame=galactocentric_frame, vcirc=vcirc, vlsr=vlsr)

#         ix = distance_ix(model_c.l, model_c.distance) & (model_c.l > 0) & (model_c.l < 10*u.deg) | True
        ix = np.ones(model_c.l.size).astype(bool)
        ax.plot(stream.pos[0][ix], stream.pos[2][ix], **style)
        
        lbd_corners = np.vstack(map(np.ravel, np.meshgrid([1.5,9.5],[26.,33.],[5.3,9.7])))
        corners_g = coord.Galactic(l=lbd_corners[0]*u.degree, b=lbd_corners[1]*u.degree, distance=lbd_corners[2]*u.kpc)
        corners = corners_g.transform_to(galactocentric_frame).cartesian.xyz.value
        
#         corners = corners.reshape(3,2,2,2)[...,0].reshape(3,4)
        # only pick 4 of the points to plot
        corners = corners[:,corners[1] > 0.5][[0,2]]
        
        # compute centroid
        cent = np.mean(corners, axis=1)
        
        # sort by polar angle
        corner_list = corners.T.tolist()
        corner_list.sort(key=lambda p: np.arctan2(p[1]-cent[1],p[0]-cent[0]))
        
        # plot polyline
        poly = mpl.patches.Polygon(corner_list, closed=True, fill=False, color='#2166AC', linestyle='-')
        ax.add_patch(poly)
        
#         ax.plot(corners[0], corners[2], ls='none', marker='o', markersize=4)
#         ax.plot(-galactocentric_frame.galcen_distance, 0., marker='o', color='y', markersize=8)
        ax.text(-galactocentric_frame.galcen_distance.value, 0., r"$\odot$", fontsize=18, ha='center', va='center')
        
        # Text
        ax.set_title(name_map[name], fontsize=20)
    
        if k == 1:
            ax.set_xlabel("$x$ [kpc]", fontsize=18)
        
#         break
#     break
    
# Axis limits
ax.set_xlim(-10,5)
ax.set_ylim(-7.5,7.5)

ax.xaxis.set_ticks([-8,-4,0,4])
ax.yaxis.set_ticks([-4,0,4])

allaxes[0,0].set_ylabel("$z$ [kpc]", fontsize=18)
allaxes[1,0].set_ylabel("$z$ [kpc]", fontsize=18)

fig.tight_layout()
    
fig.savefig(os.path.join(plotpath, "mockstream-xyz.pdf"))
fig.savefig(os.path.join(plotpath, "mockstream-xyz.png"), dpi=400)